In [1]:
import os
import time
import os.path as osp

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision.datasets import CIFAR10
from torchvision import datasets
from torchvision import transforms
import torchvision

from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
from PIL import Image
from clip import clip

In [2]:
# random seed
SEED = 1 
NUM_CLASS = 10

# Training
BATCH_SIZE = 128
NUM_EPOCHS = 30
EVAL_INTERVAL=1
SAVE_DIR = './log'

# Optimizer
LEARNING_RATE = 1e-2
MOMENTUM = 0.9
STEP=5
GAMMA=0.5

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# cifar10 transform
transform_cifar10_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_cifar10_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# 加载训练集
train_set = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_cifar10_train)
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# 加载完整测试集
test_set = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_cifar10_test)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 4, 3)  
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 8, 3)  
        self.fc1 = nn.Linear(8 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 8 * 6 * 6)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
model = ConvNet()
model.to(device)

ConvNet(
  (conv1): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=288, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()

    if epoch % EVAL_INTERVAL == 0:
        print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_dataloader)}')

print('Finished Training')

Epoch 1, Loss: 1.9281428034043373
Epoch 2, Loss: 1.6396934961723855
Epoch 3, Loss: 1.559083075169712
Epoch 4, Loss: 1.4930246303148587
Epoch 5, Loss: 1.4682737817544766
Epoch 6, Loss: 1.396606067257464
Epoch 7, Loss: 1.377064997277906
Epoch 8, Loss: 1.3581045259295217
Epoch 9, Loss: 1.353447189416422
Epoch 10, Loss: 1.3473803247027385
Epoch 11, Loss: 1.3116161323264433
Epoch 12, Loss: 1.3093953309461588
Epoch 13, Loss: 1.3017824308951493
Epoch 14, Loss: 1.2897248239163548
Epoch 15, Loss: 1.2915869932955184
Epoch 16, Loss: 1.2736687702900917
Epoch 17, Loss: 1.2681980730627505
Epoch 18, Loss: 1.2712319311888323
Epoch 19, Loss: 1.2712471372331196
Epoch 20, Loss: 1.2663163617443856
Epoch 21, Loss: 1.2555104178540848
Epoch 22, Loss: 1.2545206266291
Epoch 23, Loss: 1.2489402580749043
Epoch 24, Loss: 1.2474176229723275
Epoch 25, Loss: 1.2514717136807454
Epoch 26, Loss: 1.247416433928263
Epoch 27, Loss: 1.2479285006328007
Epoch 28, Loss: 1.241444101571427
Epoch 29, Loss: 1.2431836161772003
Epo

In [8]:
# 评估基准模型的函数
acc = 0
def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = correct / total
    return 100 * correct / total

In [9]:
# 评估基准模型
accuracy_convnet = evaluate_model(model, test_dataloader, device)
print(f'Accuracy of ConvNet on CIFAR10 is {accuracy_convnet}%')

Accuracy of ConvNet on CIFAR10 is 59.45%


In [10]:
print(f"CNN : {accuracy_convnet}%")

CNN : 59.45%


In [12]:
torch.cuda.empty_cache()